In [ ]:
# %%
! pip install pytelegrambotapi
! pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.0/245.0 kB 3.8 MB/s eta 0:00:00


In [ ]:
# %%
import telebot
import unicodedata
import re
import requests
import time
import string

In [ ]:
# %%
from transformers import pipeline


In [ ]:
# %%
API_TOKEN = 'INSIRA SEU TOKEN'
bot = telebot.TeleBot(API_TOKEN)


In [ ]:
# %%
conversation_state = {}
user_data = {}

menu = {
    "Hatch": 55.0,
    "Sedan": 85.0,
    "SUV": 100.0,
    "Wagon": 120.0,
    "Utilitário": 140.0,
    "Luxo": 200.0,
    "Esportivo": 500.0
}


def normalize_text(text):
    return unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('ASCII').lower()

@bot.message_handler(commands=['start'])
def reply_hi(message):
    chat_id = message.chat.id


    if chat_id not in conversation_state:
        welcome_message = "Seja Bem Vindo á LocAutoBots!\n"
    else:
        welcome_message = "Seja Bem Vindo á LocAutoBots!\n"


    conversation_state[chat_id] = 'aguardando_acao'
    user_data[chat_id] = {}

    bot.reply_to(message, welcome_message +
                     "Como podemos ajuda-lo hoje?\n""Digite 'ajuda' para as opções disponiveis.")


In [ ]:
# %%
@bot.message_handler(func=lambda message: True)
def process_message(message):
    chat_id = message.chat.id
    user_message = normalize_text(message.text)

    if user_message == 'localizacao':
        request_location(message)
    elif user_message == 'ajuda':
        show_help_menu(message)
    elif conversation_state.get(chat_id) == 'aguardando_acao':
        if 'alugar' in user_message or 'reservar' in user_message:
            choose_auto(message, user_message)
        elif user_message == 'auto':
            show_menu(message)
        elif user_message.capitalize() in menu:
            choose_auto(message, user_message.capitalize())
        else:
            bot.reply_to(message, "Desculpe, não entendi. Digite 'ajuda' para ver as opções!")
    elif conversation_state.get(chat_id) == 'escolhendo_auto':
        choose_auto(message, user_message)
    elif conversation_state.get(chat_id) == 'confirmando_auto':
        confirm_auto(message, user_message)
    elif conversation_state.get(chat_id) == 'definindo_quantidade':
        set_quantity(message, user_message)
    elif conversation_state.get(chat_id) == 'adicionando_mais_auto':
        add_more_auto(message, user_message)
    elif conversation_state.get(chat_id) == 'confirmando_pedido':
        confirm_order(message, user_message)
    elif conversation_state.get(chat_id) == 'help':
        if user_message == '1':
            show_menu(message)
        elif user_message == '2':
            show_about(message)
        elif user_message == '3':
            show_contact(message)
        elif user_message == '4':
            request_location(message)
        else:
            bot.reply_to(message, "Por favor, digite uma opção válida.")
        conversation_state[chat_id] = 'aguardando_acao'



In [ ]:
# %%
def show_help_menu(message):
    chat_id = message.chat.id
    help_options = "Opções de ajuda disponíveis:\n""Digite o nº da opção desejada:\n"

    help_options += "\n1. Reservas\n"
    help_options += "2. Sobre\n"
    help_options += "3. Contato\n"
    help_options += "4. Cadastro\n"
    bot.reply_to(message, help_options)
    conversation_state[chat_id] = 'help'  # Adicionado aqui

def show_about(message):
    about_info = "Sobre: Locadora AutoBots\n"
    about_info += "Somos uma locadora de automóveis especializada em fornecer veículos de qualidade para nossos clientes. Estamos comprometidos em oferecer um excelente serviço e garantir uma experiência de aluguel de automóveis agradável e conveniente. AutoBots, vamos rodar!\n"
    bot.reply_to(message, about_info)

def show_contact(message):
    contact_info = "Contato: Locadora AutoBots\n"
    contact_info += "Endereço: Rua Cybertron, 12345\n"
    contact_info += "Telefone: (11) 98800-1122\n"
    contact_info += "E-mail: autobots@locadora.com\n"
    bot.reply_to(message, contact_info)

In [ ]:
# %%
def show_menu(message):
    chat_id = message.chat.id
    menu_text = "\n".join([f"- {autos} (R$ {preco:.2f})" for autos, preco in menu.items()])
    bot.reply_to(message, f"Nossas opções de Automoveis:\n{menu_text}\nQual você gostaria de escolher?")
    conversation_state[chat_id] = 'escolhendo_auto'
    if chat_id not in user_data or 'autos' not in user_data[chat_id]:
        user_data[chat_id] = {'autos': []}



In [ ]:
# %%
def preprocess_text(text):
    # Remove pontuação
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Converter para minúsculas
    text = text.lower()
    return text

def choose_auto(message, user_message):
    chat_id = message.chat.id
    classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli")

    # Definir as palavras-chave para procurar na frase do usuário
    keywords = ["alugar", "reservar"]
    user_intent = None

    # Procurar pelas palavras-chave na frase do usuário
    for keyword in keywords:
        if keyword in user_message:
            user_intent = user_message.split(keyword)[1].strip()
            break

    if user_intent is None:
        # Se nenhuma palavra-chave for encontrada, informar ao usuário que a frase não foi entendida
        bot.reply_to(message, "Desculpe, não entendi. Por favor, use 'alugar' ou 'reservar'.")
        return


    typed_order = "" + keywords[0] + " " + user_intent


    menu_options = list(menu.keys())


    results = classifier(typed_order, candidate_labels=menu_options, hypothesis_template="O Veiculo é {}.")

    # Verificar os resultados
    print(results)

    max_score_index = results["scores"].index(max(results["scores"]))
    predicted_reserva = results["labels"][max_score_index]

    bot.reply_to(message, f"Você quis dizer {predicted_reserva}? Por favor, confirme (sim/não).")

    user_data.setdefault(chat_id, {}).setdefault('autos', {})
    user_data[chat_id]['auto'] = predicted_reserva

    conversation_state[chat_id] = 'confirmando_auto'

def confirm_auto(message, user_message):
    chat_id = message.chat.id
    if user_message == 'sim':
        auto_escolhido = user_data[chat_id]['auto']


        if not isinstance(user_data[chat_id]['autos'], dict):
            user_data[chat_id]['autos'] = {}

        user_data[chat_id]['autos'][auto_escolhido] = None
        bot.reply_to(message, f"Ok, {auto_escolhido} adicionado a sua reserva. Quantas Diárias você gostaria?")
        conversation_state[chat_id] = 'definindo_quantidade'

    elif user_message == 'nao':
        bot.reply_to(message, "Por favor, escolha um Automóvel.")
        conversation_state[chat_id] = 'escolhendo_auto'
    else:
        bot.reply_to(message, "Desculpe, não entendi. Por favor, responda com 'sim' ou 'não'.")


def add_more_auto(message, user_message):
    chat_id = message.chat.id
    if user_message is None:
        bot.reply_to(message, "Deseja reservar outro Automóvel? (sim/não)")
        conversation_state[chat_id] = 'adicionando_mais_auto'
    elif user_message == 'sim':
        show_menu(message)
    elif user_message == 'nao':
        show_order_confirmation(message)
    else:
        bot.reply_to(message, "Desculpe, não entendi. Por favor, responda com 'sim' ou 'não'.")






In [ ]:
# %%
def set_quantity(message, user_message):
    chat_id = message.chat.id
    autos = user_data[chat_id]["autos"]


    next_auto = next((auto for auto, quantity in autos.items() if quantity is None), None)

    if user_message is None:

        if next_auto:
            bot.reply_to(message, f"Quantos dias {next_auto} você gostaria de reservar?")
        else:
            bot.reply_to(message, "Você ainda não escolheu nenhum Automóvel. Digite 'automovel' para ver as opções.")
            conversation_state[chat_id] = 'aguardando_acao'
            return

        conversation_state[chat_id] = 'definindo_quantidade'
    else:
        try:
            quantidade = int(user_message)
            if quantidade > 0:

                user_data[chat_id]["autos"][next_auto] = quantidade


                next_auto = next((auto for auto, quantity in autos.items() if quantity is None), None)

                if next_auto:

                    bot.reply_to(message, f"São {next_auto} dias que você gostaria de reservar?")
                else:

                    add_more_auto(message, None)
            else:
                bot.reply_to(message, "Por favor, informe uma quantidade válida (maior que zero).")
        except ValueError:
            bot.reply_to(message, "Por favor, informe um número para a quantidade.")




In [ ]:
# %%
def show_order_confirmation(message):
    chat_id = message.chat.id
    autos_pedido = user_data[chat_id]["autos"]

    preco_total = sum(menu[d] * q for d, q in autos_pedido.items())


    pedido_resumo = "\n".join(
        [f"- {q} diárias de {d} (R$ {menu[d]*q:.2f})" for d, q in autos_pedido.items()]
    )
    bot.reply_to(
        message,
        f"Perfeito! Seu pedido:\n{pedido_resumo}\nPreço total: R$ {preco_total:.2f}\nConfirma? (sim/não)",
    )
    conversation_state[chat_id] = "confirmando_pedido"



def confirm_order(message, user_message):
    chat_id = message.chat.id
    if user_message == 'sim':
        bot.reply_to(message, "Reserva Feita! Agora, só precisamos do seu endereço para o cadastro.")
        request_location(message)
        conversation_state[chat_id] = 'aguardando_localizacao'
    else:
        bot.reply_to(message, "Reserva Cancelada. Se quiser refazer, é só me dizer!")
        conversation_state[chat_id] = 'aguardando_acao'
        user_data[chat_id] = {}



In [ ]:
from getpass import getpass

api_key = getpass('sua_chave_api')

sua_chave_api··········


In [ ]:
# %%
def request_location(message):
    chat_id = message.chat.id
    bot.send_message(
        chat_id,
        "Por favor, compartilhe sua localização para facilitar o cadastro.",
        reply_markup=telebot.types.ReplyKeyboardMarkup(
            resize_keyboard=True, one_time_keyboard=True
        ).add(
            telebot.types.KeyboardButton(
                text="Compartilhar localização", request_location=True
            )
        )
    )
    conversation_state[chat_id] = 'aguardando_localizacao'

# Função para obter o endereço a partir das coordenadas
def get_address_from_coordinates(latitude, longitude, api_key):
    url = f"https://maps.googleapis.com/maps/api/geocode/json?latlng={latitude},{longitude}&key={api_key}"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise exception for bad HTTP responses
        data = response.json()
        if data['status'] == 'OK':
            return data['results'][0]['formatted_address']
        else:
            return "Endereço não encontrado"
    except requests.exceptions.RequestException as e:
        print(f"Error making request: {e}")
        return "Endereço não encontrado devido a um erro na API"
    except requests.exceptions.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        return "Endereço não encontrado devido a um erro na resposta da API"

# Função para lidar com a localização enviada pelo usuário
@bot.message_handler(
    content_types=['location'],
    func=lambda message: conversation_state.get(message.chat.id) == 'aguardando_localizacao'
)
def handle_location_and_confirm(message):
    chat_id = message.chat.id
    latitude = message.location.latitude
    longitude = message.location.longitude
    endereco = get_address_from_coordinates(latitude, longitude, api_key)

    if endereco:
        bot.reply_to(message, f"Localização recebida! Endereço aproximado:")
        bot.send_message(chat_id, f"{endereco}")
        bot.send_message(chat_id, "Por favor, copie o endereço acima, corrija-o e inclua complementos, se necessário, e envie-o de volta:")
        conversation_state[chat_id] = 'aguardando_endereco_correto'
        bot.register_next_step_handler(message, handle_endereco_correto)
    else:
        bot.reply_to(message, "Não foi possível encontrar o endereço correspondente às coordenadas fornecidas.")


In [ ]:
# %%
def handle_endereco_correto(message):
    chat_id = message.chat.id
    endereco_correto = message.text
    user_data[chat_id]["endereco"] = endereco_correto
    bot.send_message(
        chat_id, f"Endereço atualizado:\n{endereco_correto}.\nConfirma o endereço de cadastro? (sim/não)"
    )
    conversation_state[chat_id] = 'confirmando_endereco'
    bot.register_next_step_handler(message, handle_confirmacao_endereco_message)

def handle_confirmacao_endereco(message, user_message):
    chat_id = message.chat.id
    if user_message == 'sim':
        # Confirmar o pedido
        bot.reply_to(message, "Cadastro Feito! Obrigado!")
        conversation_state[chat_id] = 'aguardando_acao'
        user_data[chat_id] = {}  # Reset user data for the next order
    else:
        # Reiniciar o processo de solicitação de endereço
        bot.reply_to(message, "Ok, vamos tentar novamente. Por favor, compartilhe sua localização:")
        request_location(message)


@bot.message_handler(func=lambda message: conversation_state.get(message.chat.id) == 'confirmando_endereco')
def handle_confirmacao_endereco_message(message):
    handle_confirmacao_endereco(message, message.text.lower())


In [ ]:
# %%
bot.polling()




{'sequence': 'alugar um barco', 'labels': ['Utilitário', 'Esportivo', 'Luxo', 'Wagon', 'Hatch', 'Sedan', 'SUV'], 'scores': [0.7833740711212158, 0.05950348824262619, 0.058453891426324844, 0.05507931858301163, 0.035537078976631165, 0.004572639241814613, 0.003479478182271123]}
{'sequence': 'alugar uma bmw', 'labels': ['Esportivo', 'Utilitário', 'Wagon', 'SUV', 'Hatch', 'Luxo', 'Sedan'], 'scores': [0.4441850483417511, 0.24458308517932892, 0.13492174446582794, 0.09451165795326233, 0.029836714267730713, 0.027710463851690292, 0.024251338094472885]}
{'sequence': 'alugar um rolls royce', 'labels': ['Utilitário', 'Luxo', 'Wagon', 'Hatch', 'Esportivo', 'SUV', 'Sedan'], 'scores': [0.48031795024871826, 0.2578819990158081, 0.17652224004268646, 0.047633495181798935, 0.020267415791749954, 0.01241132989525795, 0.004965605679899454]}
